# Playground Datasets

本项目基于

## 2.定义计算图

In [24]:
#!/usr/bin/env python
# coding:utf-8
import tensorflow as tf
from tensorflow.contrib.layers import fully_connected
import os
import dataset_g
%matplotlib inline
import matplotlib.pyplot as plt

tf.reset_default_graph()

# get data from dataset_g with relative parameters
num_samples = 500
noise = 0.05
data, label = dataset_g.get_samples(dataset_g.classify_spiral_data, num_samples, noise, False)

# Construct computation graph
X = tf.placeholder(dtype=tf.float32, shape=(None,2), name='input')
Y = tf.placeholder(dtype=tf.float32, shape=(None,1), name='label')

# define numbers of hidden layers and relative nodes
# try 6 hidden layers with following nodes  
n_hidden = [8,8,6,6,4,4]
n_output = 1
with tf.name_scope("dnn_spiral"):
    hidden = []
    hidden.append(X)
    for i in range(len(n_hidden)):
        hidden.append(fully_connected(hidden[i], n_hidden[i], tf.nn.leaky_relu, scope="hidden"+str(i+1),weights_initializer=tf.random_uniform_initializer(minval=-1, maxval=1),
                                 biases_initializer=tf.constant_initializer(0.1)))
    pred_y = fully_connected(hidden[len(n_hidden)], n_output,tf.nn.tanh, scope="output" ,weights_initializer=tf.random_uniform_initializer(minval=-0.5, maxval=0.5),
                             biases_initializer=tf.constant_initializer(0.1))

with tf.name_scope("get_loss"):
    #pred_y = tf.nn.sigmoid(logit_y)
    #xentropy = tf.nn.sigmoid_cross_entropy_with_logits(labels=Y, logits=logit_y)
    loss = tf.reduce_mean(tf.square(pred_y - Y), name="loss")

learning_rate = 0.03
with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)
    
# evaluate by accuracy
with tf.name_scope("eval"):
    concat_preds = tf.concat([tf.zeros_like(pred_y, dtype=tf.float32),pred_y],axis=1) #concat (500,1) to (500,2)
    correct = tf.nn.in_top_k(concat_preds, tf.cast(tf.clip_by_value(tf.squeeze(Y, [1]), 0., 1.), tf.int32), 1) #transfer label from -1 or 1 to 0 or 1 to be index
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    init = tf.global_variables_initializer()

## 3.执行和评估

In [26]:
# save 'loss' and 'accuracy' sequences
init = tf.global_variables_initializer()
loss_val = []
acc_val = []
n_epochs = 4000
batch_size = 30
with tf.Session() as sess:    #tf.Session和tf.Session的区别在于后者创建的会话需要手动关闭，资源不会自动释放，仍可调用。
    init.run()
    num = data.shape[0]
    print("Model: Spiral data")
    for epoch in range(n_epochs):
        for iter in range(num // batch_size):
            X_batch = data[iter*batch_size: (iter+1)*batch_size, :]
            Y_batch = label[iter*batch_size: (iter+1)*batch_size, :]
            sess.run(training_op, feed_dict={X: X_batch, Y: Y_batch})
        l, acc = sess.run([loss, accuracy], feed_dict={X: data, Y: label})
        loss_val.append([epoch, l])
        acc_val.append([epoch, acc])
        if not ((epoch) % 400):
            preds = sess.run(pred_y, feed_dict={X: data, Y: label})
            print(epoch)
            print("Prediction:\n{}, \nTrue label:\n{},"
                  " \nLoss:{:.3f}, \nAccuracy:{:.2%}".format(preds[::100], label[::100], l, acc))
    

Model: Spiral data
0
Prediction:
[[-0.3677785 ]
 [-0.5733165 ]
 [-0.11528946]
 [-0.3627441 ]
 [-0.25353688]], 
True label:
[[ 1]
 [ 1]
 [ 1]
 [-1]
 [-1]], 
Loss:1.181, 
Accuracy:50.00%
400
Prediction:
[[ 0.5938435]
 [ 0.5949684]
 [ 0.882275 ]
 [-0.9987795]
 [ 0.9918728]], 
True label:
[[ 1]
 [ 1]
 [ 1]
 [-1]
 [-1]], 
Loss:0.602, 
Accuracy:82.80%
800
Prediction:
[[ 0.73944813]
 [ 0.74226886]
 [ 1.        ]
 [-0.99540627]
 [-0.9493934 ]], 
True label:
[[ 1]
 [ 1]
 [ 1]
 [-1]
 [-1]], 
Loss:0.145, 
Accuracy:95.80%
1200
Prediction:
[[-0.6680819]
 [ 0.8861599]
 [ 1.       ]
 [-0.9478015]
 [ 1.       ]], 
True label:
[[ 1]
 [ 1]
 [ 1]
 [-1]
 [-1]], 
Loss:1.027, 
Accuracy:73.80%
1600
Prediction:
[[ 0.30264997]
 [ 0.87593716]
 [ 1.        ]
 [-0.9690696 ]
 [-0.99879396]], 
True label:
[[ 1]
 [ 1]
 [ 1]
 [-1]
 [-1]], 
Loss:0.168, 
Accuracy:96.00%
2000
Prediction:
[[ 0.8775457 ]
 [ 0.98826337]
 [ 1.        ]
 [-0.9948073 ]
 [-0.99541587]], 
True label:
[[ 1]
 [ 1]
 [ 1]
 [-1]
 [-1]], 
Loss:0.160,

In [4]:
# for test
import tensorflow as tf
from tensorflow.contrib.layers import fully_connected
import os
import dataset_g
%matplotlib inline
import matplotlib.pyplot as plt

data, label = dataset_g.get_samples(dataset_g.classify_two_gauss_data, 500, 0.1,True)
data.shape
label.shape

(500, 1)

In [17]:
hidden

[<tf.Tensor 'input:0' shape=(?, 2) dtype=float32>,
 <tf.Tensor 'dnn_spiral/hidden1/Relu:0' shape=(?, 8) dtype=float32>,
 <tf.Tensor 'dnn_spiral/hidden2/Relu:0' shape=(?, 8) dtype=float32>,
 <tf.Tensor 'dnn_spiral/hidden3/Relu:0' shape=(?, 6) dtype=float32>,
 <tf.Tensor 'dnn_spiral/hidden4/Relu:0' shape=(?, 6) dtype=float32>,
 <tf.Tensor 'dnn_spiral/hidden5/Relu:0' shape=(?, 4) dtype=float32>,
 <tf.Tensor 'dnn_spiral/hidden6/Relu:0' shape=(?, 4) dtype=float32>]

In [18]:
pred_y

<tf.Tensor 'dnn_spiral/output/Tanh:0' shape=(?, 1) dtype=float32>